In [ ]:
%matplotlib notebook

# %qtconsole

import logging
import time
from importlib import reload

from matplotlib import pyplot as plt
from matplotlib import colors, cm
import numpy as np

import qcodes

In [ ]:
%qtconsole

# Imports

In [ ]:
from pytopo.awg_sequencing import broadbean as bbtools; reload(bbtools)
from pytopo.rf.alazar import acquisition_tools; reload(acquisition_tools)
from pytopo.rf.alazar import awg_sequences; reload(awg_sequences)
from pytopo.rf.alazar import acquisition_controllers; reload(acquisition_controllers)

# Init

## initialize station, create instruments

In [ ]:
# %run -i D:/OneDrive/BF2/Code/BF2/BF2/init_station_cqed.py
%run -i D:/OneDrive/Setups/LK1/LK1_code/Code/LK1/LK1/init_station_cqed.py

In [ ]:
raw_acq = instools.create_inst(acquisition_controllers.RawAcqCtl, 
                               'raw_acq', 'alazar',
                               allocate_samples=1e9, 
                               force_new_instance=True)

acquisition_controllers.PostDemodCtl._datadtype = np.int32
post_demod_acq = instools.create_inst(acquisition_controllers.PostDemodCtl, 
                                      'post_demod_acq', 'alazar',
                                      allocate_samples=1e9, 
                                      force_new_instance=True)

post_iq_acq = instools.create_inst(acquisition_controllers.PostIQCtl, 
                                   'post_iq_acq', 'alazar',
                                   allocate_samples=1e9, 
                                   force_new_instance=True)

In [ ]:
from qcodes.instrument_drivers.tektronix.AWG5014 import Tektronix_AWG5014
from qcodes.instrument_drivers.tektronix.AWG5208 import AWG5208

if isinstance(station.awg, Tektronix_AWG5014):
    AWGTYPE = '5014'
elif isinstance(station.awg, AWG5208):
    AWGTYPE = '5208'
else:
    raise Exception('No usable AWG.')
    
print(AWGTYPE)

## Basic instrument config

In [ ]:
# adopt this for your case. different cards have different allowed ranges.
with station.alazar.syncing():
    station.alazar.channel_range1(1)
    station.alazar.channel_range2(1)

## Test sequence

In [ ]:
from pytopo.rf.alazar.awg_sequences import AlazarTestSequence

AlazarTestSequence.chan_map = {
    1 : ['pulse', 'ats_trigger', None]
}
AlazarTestSequence.chan_settings = {
    1 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
}

In [ ]:
N = 10
pulse_times = [35e-6] * N
amplitudes = np.arange(N) * 0.02 + 0.02
frequencies = [1e6] * N
phases = np.arange(N) * 2 * np.pi/N
cycle_time = 40e-6

test_seq = AlazarTestSequence(station.awg)
test_seq.wait = 'off'
test_seq.setup_awg(pulse_times=pulse_times, frequencies=frequencies, 
                   phases=phases, amplitudes=amplitudes, 
                   cycle_time=cycle_time, pre_trig_time=0.1e-6, trig_time=0.1e-6,
                   plot=True)

# Test basic controllers

## Raw data

In [ ]:
ctl = raw_acq

t_rep = 40e-6
nsamples = 3840
npts = int(1e3)
nrecords = 100
alloc_buffers = 10
SR = 1e8

t_total = npts * t_rep
nbuffers = npts // nrecords

t, data = acquisition_tools.time_acquisition(ctl, nsamples, nrecords, nbuffers, 
                                             alloc_buffers=alloc_buffers, SR=SR, t_total=t_total)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,4))
im = ax.imshow(data[0, 0, :, :, 0], cmap=cm.seismic, aspect='auto')
cb = fig.colorbar(im)

ax.set_xlabel('Sample')
ax.set_ylabel('Record')

fig, ax = plt.subplots(1, 1, figsize=(6,4))
im = ax.imshow(data[0, -1, :, :, 0], cmap=cm.seismic, aspect='auto')
cb = fig.colorbar(im)

ax.set_xlabel('Sample')
ax.set_ylabel('Record')

In [ ]:
# avgdata = data.mean(axis=0)

# fig, ax = plt.subplots(1, 1)
# ax.plot(avgdata[4, :, 0], 'o-')

In [ ]:
# fig, ax = plt.subplots(1,1)
# ax.plot(ctl.handling_times, 'o')
# ax.set_xlabel('buffer')
# ax.set_ylabel('handling time (ms)')

## Post-acq. demodulation

### Only demodulation

In [ ]:
ctl = post_demod_acq
ctl.average_buffers(True)

demod_frq = 1e6
t_rep = 40e-6
nsamples = 3840 - 64
npts = int(1e6)
nrecords = 100
alloc_buffers = 10
SR = 1e8

t_total = npts * t_rep
nbuffers = npts // nrecords

t, data = acquisition_tools.time_acquisition(ctl, nsamples, nrecords, nbuffers, 
                                             alloc_buffers=alloc_buffers, 
                                             SR=SR, t_total=t_total)

print(f'post_acquire time: {ctl.post_acquire_time:.2f} s')
print('Data shape:', data.shape)

In [ ]:
iqdata = data.mean(axis=-2)
mag = np.abs(iqdata)
phase = np.angle(iqdata, deg=True)
phase[phase < -180] += 360

# fig, ax = plt.subplots(1,1)
# ax.plot(mag[0, :, 0], 'o-')
# ax.plot(mag[-1, :, 0] + 0.01, 'o-')

# fig, ax = plt.subplots(1,1)
# ax.plot(phase[0, :, 0], 'o-')
# ax.plot(phase[-1, :, 0]+50, 'o-')

fig, (ax, bx) = plt.subplots(2, 1, sharex='all')
ax.plot(mag[0, 0, :, 0], 'o-')
bx.plot(phase[0, 0, :, 0], 'o-')
bx.set_xlabel('Record')
ax.set_ylabel('Magnitude (V)')
bx.set_ylabel('Phase (deg)')

### Extracting I/Q

In [ ]:
ctl = post_iq_acq
ctl.average_buffers(True)
ctl.buffers_per_block(None)
ctl.integration_start(1.25e-6)
ctl.integration_stop(11.25e-6)

demod_frq = 1e6
t_rep = 40e-6
nsamples = 3840 - 64
npts = int(1e5)
nrecords = 100
alloc_buffers = 10
SR = 1e8

t_total = npts * t_rep
nbuffers = npts // nrecords

t, data = acquisition_tools.time_acquisition(ctl, nsamples, nrecords, nbuffers, 
                                             alloc_buffers=alloc_buffers, 
                                             SR=SR, t_total=t_total)

print(f'post_acquire time: {ctl.post_acquire_time:.2f} s')
print('Data shape:', data.shape)

In [ ]:
mag = np.abs(data)
phase = np.angle(data, deg=True)
phase[phase < -180] += 360

# fig, ax = plt.subplots(1,1)
# ax.plot(mag[0, :, 0], 'o-')
# # ax.plot(mag[-1, :, 0] + 0.01, 'o-')

# fig, ax = plt.subplots(1,1)
# ax.plot(phase[0, :, 0], 'o-')
# # ax.plot(phase[-1, :, 0]+50, 'o-')

fig, (ax, bx) = plt.subplots(2, 1, sharex='all')
ax.plot(mag[0, 0, :, 0], 'o-')
bx.plot(phase[0, 0, :, 0], 'o-')
bx.set_xlabel('Record')
ax.set_ylabel('Magnitude (V)')
bx.set_ylabel('Phase (deg)')

### Extracting IQ and usage of buffer blocks

In [ ]:
from pytopo.rf.alazar.awg_sequences import TriggerSequence

TriggerSequence.chan_map = {
    1 : ['pulse', 'ats_trigger', None]
}
TriggerSequence.chan_settings = {
    1 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
}

trig_seq = TriggerSequence(station.awg, SR=1e7)
trig_seq.wait = 'first'
trig_seq.setup_awg(cycle_time=1e-3, debug_signal=True, ncycles=6, plot=True)

In [ ]:
if AWGTYPE == '5208':
    station.awg.ch1.awg_amplitude(1.0)
else:
    station.awg.ch1_amp(1.0)

In [ ]:
class MyPostIQCtl(acquisition_controllers.PostIQCtl):
    
    def _perform_step(self, num):        
        awg = station.awg
        if AWGTYPE == '5208':
            while awg.run_state() != 'Waiting for trigger':
                time.sleep(0.01)
        else:
            while awg.get_state() != 'Waiting for trigger':
                time.sleep(0.01)
                   
        time.sleep(0.1)
        if not (num+1) % self.buffers_per_block():
            inc = (num+1) // self.buffers_per_block()
            if AWGTYPE == '5208':
                awg.ch1.awg_amplitude(0.25 + inc * 0.05)
            else:
                station.awg.ch1_amp(0.25 + inc * 0.05)
            time.sleep(0.01)     
            
        if AWGTYPE == '5208':
            awg.force_triggerA()
        else:
            awg.force_trigger()
        
    def pre_acquire(self):
        super().pre_acquire()
        self._perform_step(0)
        self._perform_step(1)
    
    def buffer_done_callback(self, buffernum):
        nextstep = buffernum + 2
        self._perform_step(nextstep)

my_iq_acq = instools.create_inst(MyPostIQCtl, 
                                 'my_iq_acq', 'alazar',
                                 allocate_samples=1e7, 
                                 force_new_instance=True)

In [ ]:
if AWGTYPE == '5208':
    station.awg.ch1.awg_amplitude(0.25)
else:
    station.awg.ch1_amp(0.25)

ctl = my_iq_acq
ctl.average_buffers(True)
ctl.buffers_per_block(10)

demod_frq = 1e6
t_rep = 1e-3
nsamples = 100e-6 * 1e8 // 128 * 128
nbuffers = 100
nrecords = 6
alloc_buffers = 10
SR = 1e8

t_total = nbuffers * t_rep

t, data = acquisition_tools.time_acquisition(ctl, nsamples, nrecords, nbuffers, 
                                             alloc_buffers=alloc_buffers, 
                                             SR=SR, t_total=t_total)

print(f'post_acquire time: {ctl.post_acquire_time:.2f} s')
print('Data shape:', data.shape)

In [ ]:
mag = np.abs(data)
phase = np.angle(data, deg=True)
phase[phase < -180] += 360

fig, ax = plt.subplots(1, 1, figsize=(6,4))
im = ax.imshow(mag[:, 0, :, 0], cmap=cm.seismic, aspect='auto')
cb = fig.colorbar(im)

ax.set_xlabel('Record')
ax.set_ylabel('Block')

# Simplified data acquisition

## Basic convenience functions

In [ ]:
data = simple_triggered_sweep_acquisition(100, 15e-6, navgs=10, SR=1e8,
                                          pre_acq_func=start_awg_func, 
                                          post_acq_func=stop_awg_func)

mag = np.abs(data)
phase = np.angle(data, deg=True)
phase[phase < -180] += 360

fig, (ax, bx) = plt.subplots(2, 1, sharex='all')
ax.plot(mag[0, :, 0], 'o-')
bx.plot(phase[0, :, 0], 'o-')
bx.set_xlabel('Record')
ax.set_ylabel('Magnitude (V)')
bx.set_ylabel('Phase (deg)')

## End-user functions

In [ ]:
def measure_test_sequence(ncycles, navgs, cycle_len=10, cycle_time=40e-6, SR=1e8):
    awg = awg = qc.Instrument.find_instrument(qc.config['user']['instruments']['awg_name'])
    
    pulse_times = np.ones(cycle_len) * cycle_time - 3e-6
    amplitudes = np.arange(cycle_len) * 0.02 + 0.02
    frequencies = [1e6] * cycle_len
    phases = np.arange(cycle_len) * 2 * np.pi/cycle_len    
    
    awg_test_sequence.make_test_sequence(awg, pulse_times, frequencies, phases, amplitudes, 
                                         cycle_time=cycle_time, pre_trig_time=1e-6, trig_time=1e-6,
                                         plot=False, start_awg=False)
    
    data = simple_triggered_sweep_acquisition(ncycles*10, cycle_time-1e-6, navgs=10, SR=SR,
                                              pre_acq_func=start_awg_func, 
                                              post_acq_func=stop_awg_func)

    mag = np.abs(data)
    phase = np.angle(data, deg=True)
    phase[phase < -180] += 360
    
    return mag, phase

In [ ]:
mag, phase = measure_test_sequence(10, 1)

fig, (ax, bx) = plt.subplots(2, 1, sharex='all')
ax.plot(mag[0, :, 0], 'o-')
bx.plot(phase[0, :, 0], 'o-')
bx.set_xlabel('Record')
ax.set_ylabel('Magnitude (V)')
bx.set_ylabel('Phase (deg)')

# Scribbles